In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Read data

In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

# Generate X Y 

In [3]:
###
#cal_PMf  
###
u=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    u[i]=a.T

###
#obs_PMf
###
v=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_obs'][5112*i:5112*i+5112])
    v[i]=a.T

print(np.shape(u))
print(np.shape(v))

(544, 5112)
(544, 5112)


In [4]:
XRestruct_Fun=u
YRestruct_Fun=v
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(544, 5112)
(544, 5112)


In [5]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(544, 5112)
Yhat
(544, 5112)


# PCA & Linear regression

In [6]:
per = np.zeros(99)
for i in range (len(per)):
    per[i]=0.01*(i+1)
per_error = np.zeros(99)

In [7]:
for z in range (0,99):    
    pca = PCA(n_components=per[z])
    pca.fit(Xhat)
    x_pca=pca.transform(Xhat)
    
    model = LinearRegression()
    model.fit(x_pca, Yhat)
    
    """
    generate new cal
    """
    xf=np.zeros([61,5112])
    for i in range (544,605):
        a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        xf[i-544]=a
    new_xcal=pca.transform(xf)
    new_xfit=model.predict(new_xcal)

    """
    old data
    """
    old_x=np.zeros([605,5112])
    old_y=np.zeros([605,5112])
    #data_cal
    for i in range (0,605):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i]=a
    #data_obs
    for i in range (0,605):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i]=a

    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+544][j]
            old_y2[i][j]=old_y[i+544][j]
            fit_fx2[i][j]=new_xfit[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)
    
    per_error[z] = after_error2

# Result

In [8]:
print(per_error)

[7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835 7.24942835
 7.24942835 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043
 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043 7.27598043
 7.27598043 7.27598043 7.24673273 7.24673273 7.24673273 7.24673273
 7.24673273 7.24673273 6.92104963 6.92104963 6.567329   6.567329
 6.567329   6.52826873 6.52826873 6.52826873 6.5363984  6.5363984
 6.55666778 6.56834936 6.56834936 6.50331392 6.49322022 6.48831343
 6.48608337 6.41704139 6.43480476 6.38132751 6.24655463 6.28689986
 6.26565093 6.26172256 6.2806103  6.26467336 6.30293906 6.30905464
 6.3524704  6.46140981 6.49098661 6.52406675 6.41680256 6.3504059

In [13]:
print('on', np.argmin(per_error)+1, '% PCA', 'have the min MAE error :', min(per_error))

on 77 % PCA have the min MAE error : 6.246554634591224
